## Configurações Iniciais

In [1]:
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin

In [2]:
!nvidia-smi

Thu Jan 16 00:30:39 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!git clone https://github.com/NVIDIA/cuda-samples.git
%cd cuda-samples/Samples/1_Utilities/deviceQuery
!make

Cloning into 'cuda-samples'...
remote: Enumerating objects: 19507, done.
remote: Counting objects: 100% (4227/4227), done.
remote: Compressing objects: 100% (655/655), done.
remote: Total 19507 (delta 3932), reused 3572 (delta 3572), pack-reused 15280 (from 2)
Receiving objects: 100% (19507/19507), 133.82 MiB | 20.50 MiB/s, done.
Resolving deltas: 100% (17105/17105), done.
Updating files: 100% (4026/4026), done.
/content/cuda-samples/Samples/1_Utilities/deviceQuery
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery.o -c deviceQuery.cpp
/usr/local

In [4]:
! nvprof ./deviceQuery

./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

==720== NVPROF is profiling process 720, command: ./deviceQuery
Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of c

## Testes Realizados

Foram realizados 5 testes ao todo para a versão em CUDA do projeto final. Em cada teste o número de threads por bloco foi variado, com a intenção de fazer o cálculos de *speedup* e eficiência e analisar seus respectivos ganhos e/ou perdas de desempenho.
Os seguintes parâmetros foram utilizados nos testes:<br>
1. Threads por bloco: (1, 1);
2. Threads por bloco: (2, 2);
3. Threads por bloco: (4, 4);
4. Threads por bloco: (8, 8);
5. Threads por bloco: (16, 16);
6. Threads por bloco: (32, 32);
7. Threads por bloco: (64, 64);
8. Threads por bloco: (128, 128);
9. Threads por bloco: (256, 256);
10. Threads por bloco: (512, 512);
11. Threads por bloco: (1024, 1024);<br>
Foi escolhido 1024 para o máximo, pois nas especificações da GPU é mostrado que o máximo de threads suportadas por bloco é de 1024.

### Teste 1

In [49]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(1, 1);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [50]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 2.00401e-09
Interação 100 - Diferença média = 1.23248e-09
Interação 200 - Diferença média = 7.81794e-10
Interação 300 - Diferença média = 5.11528e-10
Interação 400 - Diferença média = 4.21632e-10
Tempo de execução: 7.196407 segundos
Concentração final no centro: 0.216512


### Teste 2

In [51]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(2, 2);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [52]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 2.00401e-09
Interação 100 - Diferença média = 1.23248e-09
Interação 200 - Diferença média = 7.81794e-10
Interação 300 - Diferença média = 5.11528e-10
Interação 400 - Diferença média = 4.21632e-10
Tempo de execução: 4.824912 segundos
Concentração final no centro: 0.216512


### Teste 3

In [53]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(4, 4);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [54]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 2.00401e-09
Interação 100 - Diferença média = 1.23248e-09
Interação 200 - Diferença média = 7.81794e-10
Interação 300 - Diferença média = 5.11528e-10
Interação 400 - Diferença média = 4.21632e-10
Tempo de execução: 4.354980 segundos
Concentração final no centro: 0.216512


### Teste 4

In [55]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(8, 8);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [56]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 2.00401e-09
Interação 100 - Diferença média = 1.23248e-09
Interação 200 - Diferença média = 7.81794e-10
Interação 300 - Diferença média = 5.11528e-10
Interação 400 - Diferença média = 4.21632e-10
Tempo de execução: 4.267633 segundos
Concentração final no centro: 0.216512


### Teste 5

In [57]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [58]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 2.00401e-09
Interação 100 - Diferença média = 1.23248e-09
Interação 200 - Diferença média = 7.81794e-10
Interação 300 - Diferença média = 5.11528e-10
Interação 400 - Diferença média = 4.21632e-10
Tempo de execução: 4.569233 segundos
Concentração final no centro: 0.216512


### Teste 6

In [59]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(32, 32);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [60]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 2.00401e-09
Interação 100 - Diferença média = 1.23248e-09
Interação 200 - Diferença média = 7.81794e-10
Interação 300 - Diferença média = 5.11528e-10
Interação 400 - Diferença média = 4.21632e-10
Tempo de execução: 4.446057 segundos
Concentração final no centro: 0.216512


### Teste 7

In [61]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(64, 64);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [62]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 0
Interação 100 - Diferença média = 0
Interação 200 - Diferença média = 0
Interação 300 - Diferença média = 0
Interação 400 - Diferença média = 0
Tempo de execução: 0.118731 segundos
Concentração final no centro: 1.000000


### Teste 8

In [63]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(128, 128);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [64]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 0
Interação 100 - Diferença média = 0
Interação 200 - Diferença média = 0
Interação 300 - Diferença média = 0
Interação 400 - Diferença média = 0
Tempo de execução: 0.113207 segundos
Concentração final no centro: 1.000000


### Teste 9

In [65]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(256, 256);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [66]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 0
Interação 100 - Diferença média = 0
Interação 200 - Diferença média = 0
Interação 300 - Diferença média = 0
Interação 400 - Diferença média = 0
Tempo de execução: 0.113568 segundos
Concentração final no centro: 1.000000


### Teste 10

In [67]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(512, 512);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [68]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 0
Interação 100 - Diferença média = 0
Interação 200 - Diferença média = 0
Interação 300 - Diferença média = 0
Interação 400 - Diferença média = 0
Tempo de execução: 0.122194 segundos
Concentração final no centro: 1.000000


### Teste 11

In [69]:
%%writefile diffusion.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 2000    // Tamanho da grade
#define T 500     // Número de iterações no tempo
#define D 0.1     // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

__host__ void diff_eq(double *C, double *C_new);

__global__ void diff_eq_kernel(double *C, double *C_new, int n);
__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n);

int main() {
    // Alocação dinâmica de memória para as matrizes na CPU
    double *C = (double *)malloc(N * N * sizeof(double));
    double *C_new = (double *)malloc(N * N * sizeof(double));

    if (C == NULL || C_new == NULL) {
        printf("Erro na alocação de memória.\n");
        return 1;
    }

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i * N + j] = 0.0;
            C_new[i * N + j] = 0.0;
        }
    }

    // Inicializar uma concentração alta no centro
    C[(N / 2) * N + (N / 2)] = 1.0;

    double start = clock();
    diff_eq(C, C_new);
    double end = clock();

    double time = (end - start) / CLOCKS_PER_SEC;
    printf("Tempo de execução: %f segundos\n", time);
    printf("Concentração final no centro: %f\n", C[(N / 2) * N + (N / 2)]);

    // Liberar a memória alocada
    free(C);
    free(C_new);

    return 0;
}

__host__ void diff_eq(double *C, double *C_new) {
    double *d_C, *d_C_new, *d_difmedio;
    size_t size = N * N * sizeof(double);

    // Alocando memória na GPU
    cudaMalloc((void **)&d_C, size);
    cudaMalloc((void **)&d_C_new, size);
    cudaMalloc((void **)&d_difmedio, sizeof(double));

    // Copiando os dados para a GPU
    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(1024, 1024);
    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x, (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    for (int t = 0; t < T; t++) {
        // Executando kernel de difusão
        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);

        // Inicializando diferença média na GPU
        cudaMemset(d_difmedio, 0, sizeof(double));

        // Atualizando matriz e calcular diferença média
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        // Copiando diferença média de volta para a CPU
        double difmedio;
        cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);

        if ((t % 100) == 0)
            printf("Interação %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
    }

    // Copiando os resultados de volta para o host
    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    // Liberando memória da GPU
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
}

__global__ void diff_eq_kernel(double *C, double *C_new, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n && j > 0 && j < n) {
        int idx = i * n + j;
        C_new[idx] = C[idx] + D * DELTA_T * (
            (C[(i + 1) * n + j] + C[(i - 1) * n + j] + C[i * n + (j + 1)] + C[i * n + (j - 1)] - 4 * C[idx]) / (DELTA_X * DELTA_X)
        );
    }
}

__global__ void update_kernel(double *C, double *C_new, double *difmedio, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i > 0 && i < n - 1 && j > 0 && j < n - 1) {
        int idx = i * n + j;
        atomicAdd(difmedio, fabs(C_new[idx] - C[idx]));
        C[idx] = C_new[idx];
    }
}

Overwriting diffusion.cu


In [70]:
!nvcc -arch=sm_60 -o diffusion diffusion.cu
!./diffusion

Interação 0 - Diferença média = 0
Interação 100 - Diferença média = 0
Interação 200 - Diferença média = 0
Interação 300 - Diferença média = 0
Interação 400 - Diferença média = 0
Tempo de execução: 0.113857 segundos
Concentração final no centro: 1.000000
